In [1]:
import pandas as pd
from functools import reduce

In [33]:
reporting_company_df = pd.read_csv("Reporting_company_details.csv")
investee_revenue_df = pd.read_csv("Investee_revenue_details.csv")
scope_1_scope_2_investee_emissions_df = pd.read_csv("Scope_1_and_2_emissions_of_Investee.csv")
look_up_for_scope1_and_scope2_ef_of_all_sectors = pd.read_csv("Look_up_file_for_Scope1_and_2_EF_of_sectors.csv")

category = "Investements"
equity_sub_category = "Equity Investemts"
scope = 3
Co2e_Unit = "kg"

reporting_company_df['equity_ownership'] = reporting_company_df['equity_ownership'].fillna(0)
reporting_company_df['financial_year'] = reporting_company_df['financial_year'].fillna(0)
reporting_company_df = reporting_company_df.fillna('Not Admissible')

investee_revenue_df['financial_year'] = investee_revenue_df['financial_year'].fillna(0)
investee_revenue_df['percentage_of_revenue_in_this_sector'] = investee_revenue_df['percentage_of_revenue_in_this_sector'].fillna(0)
investee_revenue_df['total_revenue'] = investee_revenue_df['total_revenue'].fillna(0)
investee_revenue_df = investee_revenue_df.fillna('Not Admissible')

scope_1_scope_2_investee_emissions_df['financial_year'] = scope_1_scope_2_investee_emissions_df['financial_year'].fillna(0)
scope_1_scope_2_investee_emissions_df['scope_1_emissions'] = scope_1_scope_2_investee_emissions_df['scope_1_emissions'].fillna(0)
scope_1_scope_2_investee_emissions_df['scope_2_emissions'] = scope_1_scope_2_investee_emissions_df['scope_2_emissions'].fillna(0)
scope_1_scope_2_investee_emissions_df = scope_1_scope_2_investee_emissions_df.fillna('Not Admissible')

look_up_for_scope1_and_scope2_ef_of_all_sectors['scope1_scope2_ef_kgco2e_per_dollar_revenue'] = look_up_for_scope1_and_scope2_ef_of_all_sectors['scope1_scope2_ef_kgco2e_per_dollar_revenue'].fillna(0)
look_up_for_scope1_and_scope2_ef_of_all_sectors = look_up_for_scope1_and_scope2_ef_of_all_sectors.fillna('Not Admissible')

input_emissions_dfs = [reporting_company_df, investee_revenue_df, investee_revenue_df, scope_1_scope_2_investee_emissions_df]
combined_emissions_input_dfs = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                                              left_index=False, right_index=False, 
                                              how='outer'), input_emissions_dfs)

combined_emissions_input_with_ef_df = pd.merge(combined_emissions_input_dfs, look_up_for_scope1_and_scope2_ef_of_all_sectors, on=['investee_sector'] , how = 'left')

temp = pd.DataFrame()

temp['total_scope1_scope2_emissions'] = combined_emissions_input_with_ef_df['scope_1_emissions'] + combined_emissions_input_with_ef_df['scope_2_emissions']
temp['total_scope1_scope2_emissions'] = temp['total_scope1_scope2_emissions'].fillna(0)
temp['investement_specific_method'] = temp['total_scope1_scope2_emissions'].mul(combined_emissions_input_with_ef_df['equity_ownership'])
temp['investement_specific_method'] = temp['investement_specific_method'].fillna(0)
temp.loc[temp['investement_specific_method'] == 0, 'average_data_method'] = (((combined_emissions_input_with_ef_df['total_revenue'].mul
                                                                            (combined_emissions_input_with_ef_df['scope1_scope2_ef_kgco2e_per_dollar_revenue'])).mul
                                                                            (combined_emissions_input_with_ef_df['percentage_of_revenue_in_this_sector'])).mul
                                                                            (combined_emissions_input_with_ef_df['equity_ownership']))
temp['average_data_method'] = temp['average_data_method'].fillna(0)
temp.loc[temp['investement_specific_method'] !=0, 'CO2eCalculationMethod'] = "Investement-Specific Method"
temp.loc[temp['average_data_method'] !=0, 'CO2eCalculationMethod'] = "Average-Data Method"
temp['EmissionCO2e'] = temp['investement_specific_method'] + temp['average_data_method']


inter_df['investor_name'] = combined_emissions_input_with_ef_df['investor_name']
inter_df['investor_region'] = combined_emissions_input_with_ef_df['investor_region']
inter_df['investee_company'] = combined_emissions_input_with_ef_df['investee_company']
inter_df['investee_sector'] = combined_emissions_input_with_ef_df['investee_sector']
inter_df['investee_region'] = combined_emissions_input_with_ef_df['investee_region']
inter_df['financial_year'] = combined_emissions_input_with_ef_df['financial_year']
inter_df['EmissionCO2e']= temp['EmissionCO2e']
inter_df['CO2eCalculationMethod'] = temp['CO2eCalculationMethod']

fin_eqt_df = pd.DataFrame()
fin_eqt_df['category_name'] = category
fin_eqt_df['sub_category_name'] = equity_sub_category
fin_eqt_df['scope'] = scope
fin_eqt_df = inter_df.groupby(['category_name','sub_category_name','scope','investor_name','investor_region','investee_company','investee_sector','investee_region','financial_year','EmissionCO2e','CO2eCalculationMethod'], as_index=False).agg({'EmissionCO2e' :'sum'})
fin_eqt_df['Co2e_Unit'] = Co2e_Unit
fin_eqt_df = fin_eqt_df.sort_values(by=['financial_year','investee_region' ])

fin_eqt_df.to_csv("final_out.csv")
print("Done")

Done
